In [1]:
%load_ext autoreload
%autoreload 2

import os
from utils import set_seed
import torch

set_seed(1234)
os.environ['HF_HOME'] = 'hf_cache' # Don't want model files in our home directory due to disk quota

torch.cuda.is_available()

In [3]:
from transformers import AutoTokenizer
import const
from model import DocRedModel
import json

MODEL_NAME = const.LUKE_LARGE_TACRED

rel2id = json.load(open('data/meta/rel2id.json'))
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
model = DocRedModel(model_name=MODEL_NAME,
                    tokenizer=tokenizer,
                    num_class=len(rel2id)).to('cuda')

/data2/nhanse02/thesis/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from utils import read_docred
import json

train_samples = read_docred(fp='/data2/nhanse02/thesis/data/train_annotated.json', rel2id=rel2id, tokenizer=tokenizer)
dev_samples = read_docred(fp='/data2/nhanse02/thesis/data/dev.json', rel2id=rel2id, tokenizer=tokenizer)

In [5]:
from torch.utils.data import DataLoader
from utils import collate_fn

train_dataloader = DataLoader(train_samples, batch_size=4, collate_fn=collate_fn, drop_last=True)
dev_dataloader = DataLoader(dev_samples, batch_size=8, collate_fn=collate_fn, drop_last=False)

In [ ]:
from train import train

train(model=model,
      train_dataloader=train_dataloader,
      dev_dataloader=dev_dataloader,
      num_epochs=1)